#Практическа работа 8 Журавлев Александр Дмитриевич ББМО-02-23

#Методы защиты от атак на модели ИИ

Защита с помощью Adversarial Training

In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.datasets import mnist
# Загрузка данных MNIST
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
# Нормализация данных
train_images = train_images / 255.0
test_images = test_images / 255.0
# Преобразование меток в one-hot encoding
train_labels = tf.keras.utils.to_categorical(train_labels, 10)
test_labels = tf.keras.utils.to_categorical(test_labels, 10)
# Уменьшаем количество обучающих изображений до 1000
train_images = train_images[:1000]
train_labels = train_labels[:1000]

# Функция FGSM атаки
def fgsm_attack(image, epsilon, gradient):
    perturbation = epsilon * np.sign(gradient)
    adversarial_image = image + perturbation
    adversarial_image = np.clip(adversarial_image, 0, 1)
    return adversarial_image
# Функция для генерации противоречивых примеров
def generate_adversarial_examples(model, images, labels, epsilon):
  adversarial_images = []
  for i in range(len(images)):
    image = tf.convert_to_tensor(images[i].reshape((1, 28, 28)))
    label = labels[i]
    if len(label.shape) > 1 and label.shape[1] > 1:
        label = np.argmax(label),
    label = tf.convert_to_tensor(label)
    with tf.GradientTape() as tape:
      tape.watch(image)
      prediction = model(image)
      loss = tf.keras.losses.categorical_crossentropy(label[None], prediction)
    gradient = tape.gradient(loss, image)
    adversarial_image = fgsm_attack(image.numpy(), epsilon, gradient.numpy())
    adversarial_images.append(adversarial_image.reshape(28, 28))
  return np.array(adversarial_images)
# Создание модели
def create_model():
  model = tf.keras.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(10, activation='softmax')])
  model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
  return model
# Обучение модели с противоречивыми примерами
def adversarial_training(model, train_images, train_labels, epsilon):
 for epoch in range(5): # Короткое обучение для демонстрации
  print(f'Epoch:{epoch}')
  for i in range(0, len(train_images), 64):
    batch_images = train_images[i:i+64]
    batch_labels = train_labels[i:i+64]
    # Генерация противоречивых примеров для текущей партии данных
    adversarial_images = generate_adversarial_examples(model, batch_images, batch_labels, epsilon)
    # Объединение оригинальных и противоречивых примеров
    combined_images = np.concatenate([batch_images, adversarial_images], axis=0)
    combined_labels = np.concatenate([batch_labels, batch_labels], axis=0)
    # Обучение на комбинированных данных
    model.train_on_batch(combined_images, combined_labels)
# Инициализация модели
model = create_model()
# Тренировка модели с защитой на противоречивых примерах
adversarial_training(model, train_images, train_labels, epsilon=0.1) # ограничил тренировочные данные для ускорения обучения
# Сохранение защищенной модели
model.save('adversarial_trained_model.h5')

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


/usr/local/lib/python3.10/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch:0


Epoch:1
Epoch:2
Epoch:3
Epoch:4


Градиентная маскировка (Gradient Masking)

In [2]:
import tensorflow as tf
from tensorflow.keras.layers import Activation
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

# Загрузка и подготовка данных:
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Нормализация данных:
train_images = train_images / 255.0
test_images = test_images / 255.0

# Преобразование меток в one-hot encoding:
train_labels = to_categorical(train_labels, 10)
test_labels = to_categorical(test_labels, 10)

# Обновление модели для градиентной маскировки:
def create_masked_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(10),
        Activation('softplus')])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Обучение модели с градиентной маскировкой:
masked_model = create_masked_model()
masked_model.fit(train_images, train_labels, epochs=5)

# Сохранение модели с градиентной маскировкой:
masked_model.save('mask_model.h5')

Epoch 1/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.8707 - loss: 0.4732
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.9600 - loss: 0.1346
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9738 - loss: 0.0900
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9802 - loss: 0.0644
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - accuracy: 0.9848 - loss: 0.0496


Регуляризация и нормализация для повышения устойчивости


In [3]:
# Модель с регуляризацией и нормализацией:
def create_regularized_model():
 model = tf.keras.Sequential([
 tf.keras.layers.Flatten(input_shape=(28, 28)),
 tf.keras.layers.Dense(128, activation='relu',
kernel_regularizer=tf.keras.regularizers.l2(0.01)),
 tf.keras.layers.Dropout(0.5),tf.keras.layers.BatchNormalization(),
 tf.keras.layers.Dense(10, activation='softmax')
 ])
 model.compile(optimizer='adam', loss='categorical_crossentropy',
metrics=['accuracy'])
 return model
# Обучение модели с регуляризацией и нормализацией:
regularized_model = create_regularized_model()
regularized_model.fit(train_images, train_labels, epochs=5)
# Сохранение модели с регуляризацией:
regularized_model.save('normal_model.h5')

Epoch 1/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - accuracy: 0.7919 - loss: 1.3415
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.8695 - loss: 0.6281
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.8663 - loss: 0.6125
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.8616 - loss: 0.6263
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.8598 - loss: 0.6184


Оценка моделей на противоречивых примерах

In [8]:
# Загрузка моделей:
models = {
    'Защищенная модель': tf.keras.models.load_model('adversarial_trained_model.h5'),
    'Нормализованная модель': tf.keras.models.load_model('normal_model.h5'),
    'Градиентная маскировка': tf.keras.models.load_model('mask_model.h5')
}

# Генерация противоречивых примеров
adversarial_test_images = generate_adversarial_examples(models['Защищенная модель'], test_images, test_labels, epsilon=0.1)

# Оценка моделей на противоречивых примерах
for model_name, model in models.items():
    test_loss, test_acc = model.evaluate(adversarial_test_images, test_labels)
    print(f'Точность ({model_name}): {test_acc}')

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.4425 - loss: 1.5915
Точность (Защищенная модель): 0.5026999711990356
313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.7128 - loss: 1.0239
Точность (Нормализованная модель): 0.7512999773025513
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7524 - loss: 0.8129
Точность (Градиентная маскировка): 0.7979000210762024
